In [ ]:
import cedalion
import cedalion.nirs
import numpy as np
import xarray as xr
import pint
import matplotlib.pyplot as p
import scipy.signal
import os.path

# Loading SNIRF data

This notebook uses a finger-tapping dataset in BIDS layout. Download it [here](https://github.com/rob-luke/BIDS-NIRS-Tapping) and point the variable `DATADIR` to its location.

In [ ]:
DATADIR = "/home/eike/Projekte/ibslab/30_dev/data/BIDS-NIRS-Tapping"



In its current implementation `read_snirf` extracts three things:
- geo is a `xr.DataArray` that stores for coordinates of labeled points (e.g. 'S1', 'D1, 'CZ')
- das is a list of `xr.DataArray` that contain the /nirs(i)/data(j)/dataTimeSeries arrays
- stim is a `pd.DataFrame` that contains event information

In [ ]:
#geo, das, stim = cedalion.io.read_snirf("/home/eike/Projekte/ibslab/30_dev/data/BIDS-NIRS-Tapping/sub-01/nirs/sub-01_task-tapping_nirs.snirf")[0]
elements = cedalion.io.read_snirf(os.path.join(DATADIR, "sub-01/nirs/sub-01_task-tapping_nirs.snirf"))
elem = elements[0] # there is only one NirsElement in this snirf file

Proposal: Use xarray's `DataArray` the main container for data in memory.

Note about the term 'channel'. I deviate from the convention that is used in SNIRF and MNE. These toolboxes consider "S1D1 760nm" and "S1D1 850nm" or "S1D1 HbO" and "S1D1 HbR" as different channels. I would propose a wording in which "S1D1" denotes a channel that comprises multiple components. These components could be for example  different wavelengths or different chromophores. In the context of source separation techniques these components will be futher split up. As will be shown, it is advantageous to keep extra dimensions for these components.


recorded time series:

In [ ]:
da = elem.data[0] # there is only one data element with amplitude data in this NIRS element. 
da

Source, detector and landmark positions:

In [ ]:
elem.geo3d

## Working with named dimensions

In [ ]:
da.mean("time")

Broadcasting works. Numpy functions accept xarrays as arguments. For some of them the output datatype is a xarray again. 

In [ ]:
od = -np.log(da/da.mean("time"))
od

## Selecting data by using coordinates

### select by value of coordinate

FIXME: support for units in coordinates is work in progress

In [ ]:
da.sel(wavelength=760.)

### Select time range

We can pass boolean masks to `.sel`

In [ ]:
da.sel(time=(10 < da.time) & (da.time < 20))

### select by regular expression

xarrays assign Pandas indexes to coodinates. Functionality to select rows in Pandas DataFrames works here, too. For example we can use Panda's `str` accessor to us string selection methods

In [ ]:
da.sel(channel=da.channel.str.match("S[2,3]D[1,2]"))

### Use on xarray to index another

The `geo` array stores for labeled points (e.g. 'S1', 'D1', 'CZ') the corresponding 3D coordinates:

In [ ]:
geo3d = elem.geo3d
geo3d

This data array has for the 'channel' dimensions multiple coordinates assigned, the channel labels (e.g. 'S1D1') but also the involved source and detector labels (e.g. 'S1', 'D1')

In [ ]:
da

We can use the coordinate arrays of `da` to index `geo3d` to get the positions of the sources of all channels

In [ ]:
geo3d.loc[da.source]

We can calculate the vector from source to detector for each channel:

In [ ]:
geo3d.loc[da.detector] - geo3d.loc[da.source]

... and find the channel distance:

In [ ]:
np.linalg.norm(geo3d.loc[da.detector] - geo3d.loc[da.source], axis=-1 )

Here three problems arise. First, `np.linalg.norm` returns a normal numpy ndarray. The coordinates are lost. Secondly, the units are lost. And finally, the `axis` parameter of `np.linalg.norm`does not know about the named dimensions and we have to use positional indexing instead.

A way to get a xarray return value is to use `xr.apply_ufunc`:

In [ ]:
tmp = geo3d.pint.dequantify() # remove units
dists = xr.apply_ufunc(np.linalg.norm, tmp.loc[da.source] - tmp.loc[da.detector], input_core_dims=[["pos"]], kwargs={"axis":-1})
dists

There are helper functions in cedalion.xrutils that work around these problems:

In [ ]:
dists = cedalion.xrutils.norm(geo3d.loc[da.source] - geo3d.loc[da.detector], dim="pos")
dists

# Stim

use pandas' DataFrame to store tabular stimulus data

In [ ]:
stim = elem.stim
# rename trial_types
stim.loc[stim.trial_type == "1.0", "trial_type"] = "control"
stim.loc[stim.trial_type == "2.0", "trial_type"] = "Tapping/Left"
stim.loc[stim.trial_type == "3.0", "trial_type"] = "Tapping/Right"
stim

Using the onset times in the `stim` DataFrame we can split the time series into epochs. This functionality could in principle be applied to any xarray that has a 'time' dimension. So it might make sense to provide this functionality as an accesor. For the moment, the `cd` (short for cedalion) has been defined an can be accessed like that. 

Note that in the epoched DataArray the 'time' dimensions has been renamed to 'reltime'. Also the trial_types have been added as coordinates for the 'epoch' dimension.

In [ ]:
epochs = da.cd.to_epochs(stim, ["Tapping/Left", "Tapping/Right"], before=10, after=30)
epochs

# Beer-Lambert

[Scott Prahl's tabulated extinction coefficients](https://omlc.org/spectra/hemoglobin/index.html) are implemented in cedalion. Use the wavelength coordinates to query them:

In [ ]:
E = cedalion.nirs.get_extinction_coefficients("prahl", da.wavelength)
E

Calculate the inverse matrix. /!\ Here the order of the dimensions matter. We need to tell xr.apply_ufunc to reverse the order of dimensions. Also xr.apply_ufunc struggles with quantified DataArrays.

In [ ]:
tmp = E.pint.dequantify()
Einv = xr.apply_ufunc(
    np.linalg.pinv, 
    tmp, 
    input_core_dims=[["chromo", "wavelength"]], 
    output_core_dims=[["wavelength", "chromo"]])
Einv

Check that Einv is the inverse of E by using numpy matrix multiplication 

In [ ]:
(E.values @ Einv.values).round(15)

Again, cedalion.xrutils has a wrapper for np.linalg.pinv that takes care of these problems:

In [ ]:
Einv = cedalion.xrutils.pinv(E)
display(Einv)
(E.values @ Einv.values).round(15)

Define an array of differential pathlengths factors

In [ ]:
dpf = xr.DataArray([6, 6], dims="wavelength", coords={"wavelength" : [760., 850.]})
dpf = dpf.pint.quantify("1") # differential path lengths factors are unitless
dpf

Calculate optical densities, divide by channel distances and dpfs. Then use matrix multiplciation to apply `Einv`. The matrix multiplication sums over the wavelength dimension. Note how the wavelength dimension automatically is replaced by the 'chromo' dimension.

In [ ]:
optical_density = - np.log( da / da.mean("time") )
conc = Einv @ (optical_density / ( dists * dpf))
conc = conc.pint.to("micromolar")
conc

Footgun: Note that xarrays `@` operator behaves differently. Here the left and right array match in two dimension and `@` sums each up, contracting the matrix to a single scalar.

In [ ]:
E @ Einv

Plot the concentration time traces. Use `.sel` to select channels and chromophore

In [ ]:
p.plot(conc.time, conc.sel(channel="S5D7", chromo="HbO"), "r-")
p.plot(conc.time, conc.sel(channel="S5D7", chromo="HbR"), "b-")
p.xlim(500,700)
for i, r in stim.iterrows():
    p.axvline(r.onset)
    p.axvline(r.onset+r.duration)

# Frequency Filter

Construct a 4th order Butterworth bandpass filter with $f_{min}=0.02\, \textrm{Hz}$ and $f_{max}=0.5\, \textrm{Hz}$. Use again `xr.apply_ufunc` to apply `scipy.signal.filtfilt` and get a xarray return value.

Again xr.apply_ufunc strips the units

In [ ]:
fny = da.cd.sampling_rate/2
b,a = scipy.signal.butter(4, (0.02/fny, 0.5/fny), "bandpass")
conc_filtered = xr.apply_ufunc(scipy.signal.filtfilt, b,a, conc)
conc_filtered

In [ ]:
conc_filtered = conc.cd.freq_filter(0.02, 0.5, 4)
conc_filtered

In [ ]:
p.plot(conc_filtered.time, conc_filtered.sel(channel="S5D7", chromo="HbO"), "r-")
p.plot(conc_filtered.time, conc_filtered.sel(channel="S5D7", chromo="HbR"), "b-")
p.xlim(500,700)
for i, r in stim.iterrows():
    p.axvline(r.onset)
    p.axvline(r.onset+r.duration)

# Epochs

In [ ]:
conc_epochs = conc_filtered.cd.to_epochs(stim, ["Tapping/Left", "Tapping/Right"], before=5, after=20)

In [ ]:
conc_epochs

# Block averages

To calculate average responses we need to calculate the baseline before each stimulus and subtract it. The time samples beloning to the baseline are easily selected by `conc_epochs.reltime < 0`

In [ ]:
baseline = conc_epochs.sel(reltime=(conc_epochs.reltime < 0)).mean("reltime")
conc_epochs_blcorrected = conc_epochs - baseline

Now all epochs belonging to a trial_type need to be grouped and averaged. Xarray's `groupby` operation makes that easy:

In [ ]:
blockaverage = conc_epochs_blcorrected.groupby("trial_type").mean("epoch")
blockaverage

### Plot the results

In [ ]:
f,ax = p.subplots(5,6, figsize=(12,10))
ax = ax.flatten()
for i_ch, ch in enumerate(conc_epochs_blcorrected.channel):
    for ls, trial_type in zip(["-", "--"], blockaverage.trial_type):
        #for i_epoch in range(epochs.shape[0]):
        #    ax[i_ch].plot(conc_epochs_blcorrected.reltime, conc_epochs_blcorrected.loc[i_epoch, "HbO", ch, :], "r-", alpha=.1)
        #    ax[i_ch].plot(conc_epochs_blcorrected.reltime, conc_epochs_blcorrected.loc[i_epoch, "HbR", ch, :], "b-", alpha=.1)
    
        ax[i_ch].plot(blockaverage.reltime, blockaverage.sel(chromo="HbO", trial_type=trial_type, channel=ch), "r", lw=2, ls=ls)
        ax[i_ch].plot(blockaverage.reltime, blockaverage.sel(chromo="HbR", trial_type=trial_type, channel=ch), "b", lw=2, ls=ls)
        ax[i_ch].grid(1)
        ax[i_ch].set_title(ch.values)
        ax[i_ch].set_ylim(-.2, .25)
    
p.tight_layout()

# sklearn
Evaluate the interplay of xarray and sklearn

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score

Start from baseline-corrected, epoched concentration data. 

In [ ]:
conc_epochs_blcorrected

We don't need the baseline samples so remove them:

In [ ]:
conc_epochs_blcorrected_nobl = conc_epochs_blcorrected.sel(reltime=conc_epochs_blcorrected.reltime >=0)
conc_epochs_blcorrected_nobl

In [ ]:
conc_epochs_blcorrected_nobl.channel

sklearn estimator and transforms expect datasets in the form of an 2D array X with shape (n_samples, n_features). We can transform our 4D DataArray into this shape by stacking 3 dimensions together. Also create an index for the epoch dimension, which did not have one so far.

In [ ]:
X = conc_epochs_blcorrected_nobl.stack(features=["chromo", "channel", "reltime"])
X = X.set_xindex("trial_type")
X

For a classification taks we need an array with class labels, typically called `y`. We can use sklearn's LabelEncoder to derive labels from the trial_type coordinates:

In [ ]:
y = xr.apply_ufunc(LabelEncoder().fit_transform, X.trial_type)

In [ ]:
display(y)
display(y.sel(trial_type="Tapping/Left"))
display(y.sel(trial_type="Tapping/Right"))

Sklearn's train_test_split works on xarrays and does return xarrays. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y)

In [ ]:
display(X_train.groupby("trial_type").count().rename("train"))
display(X_test.groupby("trial_type").count().rename("test"))

Train a LDA classifier and use it to predict labels. We need to use `xr.apply_ufunc` if we want to have an xarray.

In [ ]:
clf = LinearDiscriminantAnalysis(n_components=1).fit(X_train, y_train)

In [ ]:
y_pred = xr.apply_ufunc(clf.predict, X_test, input_core_dims=[["features"]])

In [ ]:
accuracy_score(y_test, y_pred)

The advantage here is that when X and y are still xarrays we still have access to the coordinate axes. That means for example, that we can still use them to select samples:

In [ ]:
p.figure()
bincount, bins, _ = p.hist(clf.decision_function(X_train.sel(trial_type="Tapping/Left")), alpha=.5, fc="r")
bincount, bins, _ = p.hist(clf.decision_function(X_train.sel(trial_type="Tapping/Right")), bins, alpha=.5, fc="g")

p.figure()
bincount, bins, _ = p.hist(clf.decision_function(X_test.sel(trial_type="Tapping/Left")), alpha=.5, fc="r")
bincount, bins, _ = p.hist(clf.decision_function(X_test.sel(trial_type="Tapping/Right")), bins, alpha=.5, fc="g")



Finally, test to use cross-validation while training our classifier:

In [ ]:
cross_validate(clf, X,y)